# How to run reV using HSDS
## Running reV Gen+Econ

[reV Gen](https://nrel.github.io/reV/reV/reV.generation.generation.html#reV.generation.generation.Gen) and [reV Econ](https://nrel.github.io/reV/reV/reV.econ.econ.html#reV.econ.econ.Econ) can be run locally using the [HDF Groups](https://www.hdfgroup.org) Highly Scalable Distributed Service (HSDS) to access resource .h5 file stored in the cloud (currenly on Amazon Web Services -- AWS).

reV use [PySAM](https://pysam.readthedocs.io/en/latest/) to
compute technologically specific capcity factor means and profiles (`Gen`) and mean levelized cost of energy (`Econ`). `reV`
uses ``SAM`` technology terms and input configuration files.


NOTE: In all of these examples, the ``sam_file`` input points to files in
the reV test directory [`TESTDATADIR`](https://github.com/NREL/reV/tree/master/tests/data) that may not be copied in your install. You may want to download the relevant SAM system configs from that directory and point the ``sam_file`` variable to the correct filepath on your computer.

### Using HSDS

The Highly Scalable Distributed Service (HSDS) is a cloud optimized API to enable access to .h5 files hosted on [AWS](https://registry.opendata.aws). The HSDS software was developed by the [HDF Group](https://www.hdfgroup.org/) and is hosted on Amazon Web Services (AWS) using a combination of EC2 (Elastic Compute) and S3 (Scalable Storage Service). You can read more about the HSDS service [in this slide deck](https://www.slideshare.net/HDFEOS/hdf-cloud-services).


#### Setting up HSDS

To get started install the h5pyd library:

```bash
pip install h5pyd
```

Next, configure h5pyd by running ``hsconfigure`` from the command line, or by
creating a configuration file at ``~/.hscfg``:

```bash
hsconfigure

hs_endpoint = https://developer.nrel.gov/api/hsds
hs_username =
hs_password =
hs_api_key = 3K3JQbjZmWctY0xmIfSYvYgtIcM3CN0cb1Y2w9bf
```

**The example API key here is for demonstration and is rate-limited per IP. To
get your own API key, visit https://developer.nrel.gov/signup/**

**Please note that our HSDS service is for demonstration purposes only, if you
would like to use HSDS for production runs of reV please setup your own
service: https://github.com/HDFGroup/hsds and point it to our public HSDS
bucket: s3://nrel-pds-hsds**

#### Using HSDS with reV

Once h5pyd has been installed and configured, [rex](https://github.com/nrel/rex) can pull data directly from AWS using [HSDS](https://github.com/NREL/hsds-examples)

To access the resource data used by reV (NSRDB or WTK) you have to turn on the
``hsds`` flag in the `rex`'s [resource handlers](https://nrel.github.io/rex/rex/rex.renewable_resource.html):

In [ ]:
import rex

nsrdb_file = '/nrel/nsrdb/v3/nsrdb_2013.h5'
with rex.Resource(nsrdb_file, hsds=True) as f:
    meta_data = f.meta
    time_index = f.time_index

display(meta.head())
diplay(time_index)

#### windpower

Compute wind capacity factors for a given set of latitude and longitude
coordinates:

In [17]:
import os
import numpy as np
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

lat_lons = np.array([[ 41.75, -73.66],
                     [ 40.05, -73.74],
                     [ 40.97, -73.78],
                     [ 41.65, -73.74],
                     [ 40.25, -73.7 ],
                     [ 41.05, -73.78]])

res_file = '/nrel/wtk/conus/wtk_conus_2012.h5'  # HSDS 'file' path
sam_file = os.path.join(TESTDATADIR, 'SAM/wind_gen_standard_losses_0.json')

pp = ProjectPoints.lat_lon_coords(lat_lons, res_file, sam_file)
gen = Gen.reV_run('windpower', pp, sam_file, res_file,
                  max_workers=1, out_fpath=None,
                  output_request=('cf_mean', 'cf_profile'))
display(gen.out['cf_profile'])

/Users/mrossol/Git_Repos/reV/reV/config/project_points.py:576: UserWarning: WARNING: points are not in sequential order and will be sorted! The original order is being preserved under column "points_order"
  warn(msg)
/Users/mrossol/Git_Repos/reV/reV/SAM/version_checker.py:101: PySAMVersionWarning: It appears old SAM v1 keys are being used. Updated key "wind_farm_losses_percent" to "turb_generic_loss".
  warn(m, PySAMVersionWarning)


KeyboardInterrupt: 

#### pvwatts

NOTE: ``pvwattsv5`` and ``pvwattsv7`` are both available from reV.

Compute pv mean capacity factors and mean levelized cost of energy for all resource gids in New York county New York:

In [4]:
import os
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

regions = {'New York': 'county'}

res_file = '/nrel/nsrdb/v3/nsrdb_2012.h5'  # HSDS 'file' path
sam_file = os.path.join(TESTDATADIR, 'SAM/naris_pv_1axis_inv13.json')

pp = ProjectPoints.regions(regions, res_file, sam_file)
gen = Gen.reV_run('pvwattsv5', pp, sam_file, res_file,
                  max_workers=1, out_fpath=None,
                  output_request=('cf_mean', 'lcoe_fcr'))
display(gen.out['cf_mean'])
display(gen.out['lcoe_fcr'])